In [1]:
#Load dependencies
import os
import pandas as pd

In [2]:
#Define data files to import
consumptionfile='../Raw Data Files/EIA Annual Electricity Fuel Consumption by State.xls'
generationfile='../Raw Data Files/EIA Annual Electricity Generation by State.xls'
coolheatfile='../Raw Data Files/EIA Cooling-Heating Degree Days by Region.csv'
emissionfile='../Raw Data Files/EIA Emission Annual Data by State.xls'
greenhousefile='../Raw Data Files/ghgp_data_by_year.xlsx'

In [3]:
#Read data files into Panda dataframes
tempconsumption=pd.ExcelFile(consumptionfile)
tempgeneration=pd.ExcelFile(generationfile)
coolheat=pd.read_csv(coolheatfile,header=4)
tempemission=pd.ExcelFile(emissionfile)
tempgreenhouse=pd.ExcelFile(greenhousefile)
consumption=tempconsumption.parse('Consumption_1990 - 2018 Final',skiprows=1,header=0)
generation=tempgeneration.parse('Net_Generation_1990-2018 Final',skiprows=1,header=0)
emission=tempemission.parse('State Emissions',header=0)
greenhouse=tempgreenhouse.parse('Direct Emitters',skiprows=3,header=0)

In [4]:
#Rename columns of dataframes for consistency and simplicity
consumption.columns=['year','state','producer type','energy source','consumption']
generation.columns=['year','state','producer type','energy source','generation (mwh)']
coolheat.columns=['year','U.S. cooling degree days','U.S. heating degree days','Pacific heating degree days',
                  'Mountain heating degree days','West South Central heating degree days',
                  'East South Central heating degree days','South Atlantic heating degree days',
                  'West North Central heating degree days','East North Central heating degree days',
                  'Middle Atlantic heating degree days','New England heating degree days',
                  'Pacific cooling degree days','Mountain cooling degree days','West South Central cooling degree days',
                  'East South Central cooling degree days','South Atlantic cooling degree days',
                  'West North Central cooling degree days','East North Central cooling degree days',
                  'Middle Atlantic cooling degree days','New England cooling degree days']
emission.columns=['year','state','producer type','energy source','CO2 (MT)','SO2 (MT)','NOx (MT)']
greenhouse.columns=['facility id','frs id','facility name','city','state','zip','address','county',
                    'latitude','longitude','naics code','industry type (subparts)','industry type (sectors)',
                    '2018 direct emissions','2017 direct emissions','2016 direct emissions','2015 direct emissions',
                    '2014 direct emissions','2013 direct emissions','2012 direct emissions','2011 direct emissions']

In [5]:
consumption.head(10)

,year,state,producer type,energy source,consumption
0,1990,AK,Total Electric Power Industry,Coal (Short Tons),404871
1,1990,AK,Total Electric Power Industry,Petroleum (Barrels),961837
2,1990,AK,Total Electric Power Industry,Natural Gas (Mcf),42764948
3,1990,AK,"Electric Generators, Electric Utilities",Coal (Short Tons),290182
4,1990,AK,"Electric Generators, Electric Utilities",Petroleum (Barrels),657706
5,1990,AK,"Electric Generators, Electric Utilities",Natural Gas (Mcf),34366142
6,1990,AK,"Combined Heat and Power, Commercial Power",Coal (Short Tons),114689
7,1990,AK,"Combined Heat and Power, Commercial Power",Petroleum (Barrels),112409
8,1990,AK,"Combined Heat and Power, Industrial Power",Petroleum (Barrels),191722
9,1990,AK,"Combined Heat and Power, Industrial Power",Natural Gas (Mcf),8398806


In [6]:
generation.head(10)

,year,state,producer type,energy source,generation (mwh)
0,1990,AK,Total Electric Power Industry,Total,5599506.0
1,1990,AK,Total Electric Power Industry,Coal,510573.0
2,1990,AK,Total Electric Power Industry,Hydroelectric Conventional,974521.0
3,1990,AK,Total Electric Power Industry,Natural Gas,3466261.0
4,1990,AK,Total Electric Power Industry,Petroleum,497116.0
5,1990,AK,Total Electric Power Industry,Wind,0.0
6,1990,AK,Total Electric Power Industry,Wood and Wood Derived Fuels,151035.0
7,1990,AK,"Electric Generators, Electric Utilities",Total,4493024.0
8,1990,AK,"Electric Generators, Electric Utilities",Coal,311960.0
9,1990,AK,"Electric Generators, Electric Utilities",Hydroelectric Conventional,974521.0


In [7]:
coolheat.head(10)

,year,U.S. cooling degree days,U.S. heating degree days,Pacific heating degree days,Mountain heating degree days,West South Central heating degree days,East South Central heating degree days,South Atlantic heating degree days,West North Central heating degree days,East North Central heating degree days,...,New England heating degree days,Pacific cooling degree days,Mountain cooling degree days,West South Central cooling degree days,East South Central cooling degree days,South Atlantic cooling degree days,West North Central cooling degree days,East North Central cooling degree days,Middle Atlantic cooling degree days,New England cooling degree days
0,2021,1401.0,4178.6,3320.2,4801.8,2031.2,3414.2,2590.8,6517.7,6259.9,...,6260.1,849.1,1477.0,2684.2,1657.0,2186.6,929.9,748.1,707.2,505.2
1,2020,1405.8,4045.5,3277.8,4760.7,1887.5,3217.4,2417.5,6428.2,6038.2,...,6142.2,849.2,1475.9,2697.4,1657.5,2213.0,930.2,748.1,707.0,505.2
2,2019,1499.3,4303.3,3548.1,5302.2,2147.3,3181.8,2380.8,7073.3,6435.6,...,6520.1,835.1,1400.4,2754.4,1875.1,2513.2,951.9,827.5,784.6,534.6
3,2018,1585.3,4277.9,3169.5,4802.6,2251.8,3478.8,2626.4,6969.0,6433.5,...,6326.2,1002.2,1573.4,2856.6,1928.2,2418.8,1134.1,972.1,885.5,667.0
4,2017,1428.5,3827.6,3186.8,4567.7,1581.8,2835.4,2231.3,5997.3,5683.6,...,6037.8,1053.1,1548.0,2718.0,1585.2,2254.3,910.7,708.7,660.6,450.4
5,2016,1558.5,3878.3,3031.1,4619.0,1752.1,3093.6,2456.0,5786.3,5700.8,...,5929.5,928.4,1497.3,2882.4,1957.0,2412.3,1072.8,957.5,887.8,625.7
6,2015,1488.2,4086.7,2899.4,4600.2,2087.1,3222.0,2488.1,6088.6,6165.5,...,6521.5,1067.2,1478.7,2741.2,1717.9,2404.3,941.9,729.2,803.8,555.2
7,2014,1298.9,4552.1,2776.3,4743.7,2422.8,3933.4,2952.7,7303.8,7195.7,...,6677.1,1069.1,1430.7,2474.7,1492.9,2007.3,812.6,609.5,595.9,419.5
8,2013,1307.0,4466.1,3365.1,5273.2,2325.9,3649.4,2729.2,7134.8,6622.7,...,6427.6,889.8,1461.5,2536.1,1439.6,2007.0,891.3,688.9,681.1,539.9
9,2012,1495.5,3769.7,3414.2,4575.3,1650.7,2876.4,2299.3,5516.3,5357.0,...,5564.0,918.0,1571.3,2914.5,1760.5,2167.9,1221.1,973.6,814.5,562.9


In [8]:
emission.head(10)

,year,state,producer type,energy source,CO2 (MT),SO2 (MT),NOx (MT)
0,1990,AK,Commercial Cogen,All Sources,824004,13198,3011
1,1990,AK,Commercial Cogen,Coal,821929,13191,3009
2,1990,AK,Commercial Cogen,Petroleum,2075,6,2
3,1990,AK,Commercial Non-Cogen,All Sources,0,149,42
4,1990,AK,Commercial Non-Cogen,Petroleum,0,149,42
5,1990,AK,Electric Utility,All Sources,2814130,2403,7837
6,1990,AK,Electric Utility,Coal,646430,832,2881
7,1990,AK,Electric Utility,Natural Gas,1886585,9,4364
8,1990,AK,Electric Utility,Petroleum,281115,1562,592
9,1990,AK,Industrial Cogen,All Sources,176547,2968,791


In [8]:
greenhouse.head(10)

,facility id,frs id,facility name,city,state,zip,address,county,latitude,longitude,...,industry type (subparts),industry type (sectors),2018 direct emissions,2017 direct emissions,2016 direct emissions,2015 direct emissions,2014 direct emissions,2013 direct emissions,2012 direct emissions,2011 direct emissions
0,1000001,1.100005e+11,PSE Ferndale Generating Station,FERNDALE,WA,98248,5105 LAKE TERRELL ROAD,WHATCOM COUNTY,48.828707,-122.685533,...,D,Power Plants,302529.480,350890.100,354145.488,405725.952,333193.564,395314.784,14719.108,35878.164
1,1000002,1.100412e+11,Ardagh Glass Inc. (Dunkirk),DUNKIRK,IN,47336,524 E. CENTER STREET,JAY COUNTY,40.371053,-85.198134,...,"C,N",Minerals,110511.712,115937.540,114530.004,114583.804,114959.652,103822.920,111294.548,109863.600
2,1000003,1.100015e+11,Ardagh Glass Inc. (Henderson),Henderson,NC,27537,620 Facet Road,VANCE COUNTY,36.291066,-78.391883,...,"C,N",Minerals,79393.210,80219.128,74813.296,80976.036,81003.988,80535.232,74324.196,77199.184
3,1000004,1.100008e+11,Ardagh Glass Inc. (Lincoln),LINCOLN,IL,62656,1200 NORTH LOGAN STREET,LOGAN COUNTY,40.157643,-89.353796,...,"C,N",Minerals,55547.748,57894.492,54780.848,59954.736,57837.096,59600.496,60085.996,52494.612
4,1000005,1.100005e+11,Ardagh Glass Inc. (Madera),MADERA,CA,93637,24441 AVENUE 12 & ROAD 24 1/2,MADERA COUNTY,36.923750,-120.104120,...,"C,N",Minerals,83863.020,82451.324,79708.576,81133.624,74754.144,78249.768,82149.608,75969.472
5,1000006,1.100008e+11,Ardagh Glass Inc. (Milford),MILFORD,MA,1757,1 NATIONAL STREET,WORCESTER,42.127778,-71.510556,...,"C,N",Minerals,9745.016,37320.220,33116.336,41248.504,42313.908,42648.156,45293.706,43434.408
6,1000007,1.100070e+11,AMERICAN GYPSUM - BERNALILLO (WALLBOARD) PLANT,BERNALILLO,NM,87004,1000 N HILL RD,SANDOVAL,35.329352,-106.525777,...,C,Minerals,10859.360,4145.684,0.000,0.000,0.000,0.000,0.000,0.000
7,1000010,1.100139e+11,SANTA ROSA CENTRAL LANDFILL,MILTON,FL,32583,6337 DELISA RD.,SANTA ROSA COUNTY,30.580400,-87.062100,...,HH,Waste,125328.250,107562.000,106339.750,101770.000,107609.500,109206.000,102640.000,98143.350
8,1000015,1.100005e+11,"TYSON FARMS, INC.",WILKESBORO,NC,28697,704 FACTORY STREET,WILKES COUNTY,36.143889,-81.163333,...,C,Other,26683.896,27070.946,25592.004,28135.430,28909.070,29057.040,25292.904,25076.254
9,1000016,1.100015e+11,TYSON FARMS INCORPORATED HARMONY,HARMONY,NC,28634,501 SHEFFIELD ROAD,IREDELL COUNTY,35.954580,-80.721080,...,C,Other,36535.312,35015.368,35333.116,32337.728,29817.888,29325.540,49787.358,50407.950
